Below codes is for Mounting Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os

Steps Involved In training ResNet::

-paths of training,testing and validation set were identified.

In [0]:
train_folder= "/content/drive/My Drive/Mnist/Train"
val_folder = "/content/drive/My Drive/Mnist/Val"
test_folder = "/content/drive/My Drive/Mnist/Test"

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


Size of image specified as per the input requirement of Model

In [0]:
image_size = 224
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.models import load_model
from keras.layers import GlobalAveragePooling2D
from keras.utils import multi_gpu_model

Modifying ResNet so that it can work on grayscale images having four classes


In [0]:
def modified_resnet():
    resnet = ResNet50(weights='imagenet', include_top=False)
    
    result = resnet.output
    result = GlobalAveragePooling2D()(result)
    result = Dense(512, activation='relu')(result)
    
    predictions = Dense(4, activation='sigmoid')(result)
    
    transfer_model = Model(inputs=resnet.input, outputs=predictions)
    
    return transfer_model

cnn = modified_resnet() 
weights_path = '/content/modified_resnet.h5'
#from keras.models import model_from_json
cnn.load_weights('/content/my_model_weights.h5')

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Below two blocks represent the lading and saving of resnet weights for CNN model

In [0]:
cnn.save_weights('my_model_weights.h5')


In [0]:
cnn.load_weights('my_model_weights.h5')

In [0]:
#cnn.summary()
cnn.save("modified_resnet.h5")
cnn = load_model(weights_path)
#cnn = model_from_json(json_string)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Then Some Data Augmentation was done for making the value of pixel between 0 and 1 so that the training procedure can be fast.


In [0]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range=20,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip = True)

In [0]:
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

 assigning value for Batch Size and Epoch

In [0]:
train_batchsize = 10
val_batchsize = 10
test_batchsize = 1

Train ,test and validation were generated by the following code:
flow_from_directory::Takes the path to a directory, and generates batches of augmented/normalized data. 


In [0]:
train_generator = train_datagen.flow_from_directory(train_folder,
                                                  target_size = (image_size, image_size),
                                                  batch_size = train_batchsize,
                                                  class_mode = 'categorical')

validation_generator = val_datagen.flow_from_directory(val_folder,
     target_size=(image_size, image_size),
     batch_size=val_batchsize,
     class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_folder,
                                            target_size = (image_size, image_size),
                                            batch_size = test_batchsize,
                                            class_mode = 'categorical',
                                            shuffle = False)

Found 2000 images belonging to 4 classes.
Found 1000 images belonging to 4 classes.
Found 800 images belonging to 4 classes.


In [0]:
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

Model Compilation

In [0]:
cnn.compile(loss='binary_crossentropy', 
            optimizer=keras.optimizers.Adam(),
            metrics=['accuracy'])

Training/Fitting od Model

In [0]:
cnn_model = cnn.fit_generator(train_generator,
                        epochs = 10,
                        steps_per_epoch=train_generator.samples/train_generator.batch_size,
                        validation_steps=validation_generator.samples/validation_generator.batch_size,
                        validation_data = validation_generator,
                        
                        verbose=1)

Epoch 1/10
200/200 [==============================] - 958s 5s/step - loss: 0.1279 - accuracy: 0.9681 - val_loss: 1.3200 - val_accuracy: 0.6250
Epoch 2/10
200/200 [==============================] - 82s 409ms/step - loss: 0.0562 - accuracy: 0.9844 - val_loss: 0.7892 - val_accuracy: 0.6250
Epoch 3/10
200/200 [==============================] - 82s 408ms/step - loss: 0.0218 - accuracy: 0.9933 - val_loss: 2.1521 - val_accuracy: 0.5913
Epoch 4/10
200/200 [==============================] - 82s 408ms/step - loss: 0.0165 - accuracy: 0.9939 - val_loss: 1.8635 - val_accuracy: 0.7575
Epoch 5/10
200/200 [==============================] - 81s 407ms/step - loss: 0.0195 - accuracy: 0.9934 - val_loss: 9.9217e-04 - val_accuracy: 1.0000
Epoch 6/10
200/200 [==============================] - 81s 407ms/step - loss: 0.0125 - accuracy: 0.9958 - val_loss: 0.0045 - val_accuracy: 1.0000
Epoch 7/10
200/200 [==============================] - 81s 407ms/step - loss: 0.0073 - accuracy: 0.9970 - val_loss: 1.6466 - val_

Evaluating Testing accuray on test data(containing 50% noise)

In [0]:
test_accu = cnn.evaluate_generator(test_generator, steps = test_generator.samples/test_generator.batch_size, verbose=1)

900/900 [==============================] - 276s 307ms/step


In [0]:
print('The testing accuracy for 50% is :',test_accu[1]*100, '%')
print('The testing accuracy for 10%(trained on 50%) :',test_accu_10[1]*100, '%')
print('The testing accuracy for 20%(trained on 50%):',test_accu_20[1]*100, '%')
print('The testing accuracy for 30%(trained on 50%) :',test_accu_30[1]*100, '%')
print('The testing accuracy for 40%(trained on 50%):',test_accu_40[1]*100, '%')
print('The testing accuracy for 60%(trained on 50%):',test_accu_60[1]*100, '%')
print('The testing accuracy for 70%(trained on 50%):',test_accu_70[1]*100, '%')

The testing accuracy for 50% is : 99.97222423553467 %
The testing accuracy for 10%(trained on 50%) : 74.84375238418579 %
The testing accuracy for 20%(trained on 50%): 86.93749904632568 %
The testing accuracy for 30%(trained on 50%) : 99.40624833106995 %
The testing accuracy for 40%(trained on 50%): 99.75000023841858 %
The testing accuracy for 60%(trained on 50%): 98.06249737739563 %


Evaluating Testing accuray on test data(containing 10% noise)

In [0]:
test_accu_10 = cnn.evaluate_generator(test_generator, steps = test_generator.samples/test_generator.batch_size, verbose=1)


800/800 [==============================] - 314s 392ms/step


Evaluating Testing accuray on test data(containing 20% noise)

In [0]:
test_accu_20 = cnn.evaluate_generator(test_generator, steps = test_generator.samples/test_generator.batch_size, verbose=1)


800/800 [==============================] - 274s 342ms/step


Evaluating Testing accuray on test data(containing 30% noise)

In [0]:
test_accu_30 = cnn.evaluate_generator(test_generator, steps = test_generator.samples/test_generator.batch_size, verbose=1)

800/800 [==============================] - 290s 362ms/step


Evaluating Testing accuray on test data(containing 40% noise)

In [0]:
test_accu_40 = cnn.evaluate_generator(test_generator, steps = test_generator.samples/test_generator.batch_size, verbose=1)


800/800 [==============================] - 283s 354ms/step


Evaluating Testing accuray on test data(containing 60% noise)

In [0]:
test_accu_60 = cnn.evaluate_generator(test_generator, steps = test_generator.samples/test_generator.batch_size, verbose=1)


800/800 [==============================] - 17s 22ms/step


for predicting probability of every image in a folder

In [0]:
y_pr=cnn.predict_generator(test_generator, steps = test_generator.samples/test_generator.batch_size, verbose=1)

900/900 [==============================] - 20s 22ms/step


Prediction of every image in the folder

In [0]:
print(y_pr)

[[9.9999809e-01 1.8899618e-07 6.3366262e-10 8.7637630e-08]
 [9.9971908e-01 2.7976986e-07 5.8427347e-09 9.0602596e-05]
 [9.9999928e-01 2.8533725e-08 7.7200735e-10 3.7617131e-08]
 ...
 [3.8825739e-12 1.6794593e-09 3.2824544e-06 9.9998963e-01]
 [9.0839803e-15 4.0473462e-12 2.0427267e-06 9.9999297e-01]
 [1.8705121e-13 3.2559774e-11 1.9622037e-05 9.9996030e-01]]
